In [1]:
import sys
sys.path.append('..')
import os
import sklearn.metrics.cluster
import sklearn.cluster
import seaborn as sns
import scanpy as sc
import sklearn as sk
from sklearn.neighbors import NearestNeighbors
import matplotlib as mpl
from matplotlib import pyplot as plt
import skdim
import seaborn as sns
import time
import numpy as np
import pandas as pd
import pickle
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages
import utils
import anndata2ri
import gc
import benchmark_common_functions
from itertools import product

anndata2ri.activate()
ro.numpy2ri.activate()
readRDS = ro.r['readRDS']
%load_ext rpy2.ipython

# Run processed datasets

In [2]:
base_folder1 = '../data/benchmark_sensitivity_frontiers/Benchmarking_Clustering_Methods_scRNAseq/raw_data/Rdata_qc_filt_norm/'
base_folder2 = '../data/benchmark_sensitivity_frontiers/Benchmarking_Clustering_Methods_scRNAseq/norm_data/Rdata_qc_filt_hvg/'
get_fnames = lambda base_folder: [os.path.join(base_folder, fname) for fname in os.listdir(base_folder)]
fnames = get_fnames(base_folder1)+get_fnames(base_folder2)

resdir = '../results/benchmark_sensitivity_frontiers/real_data/'

get_res_path = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn{n_neighbors}_seed{seed}.pkl'

In [3]:
#fixed params
do_log = True
do_pca = True
n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = (None,)
norm_scale = (None,)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))

In [4]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for fname in fnames:
        if get_res_path(fname).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname.split('/')[-1])
        x = ro.r.load(fname)
        adata = ro.r['sce'] 
        if 'kolodziejczyk' in fname:
            adata.obs['Group']=adata.obs['cell_type1']
            
        results_dict = benchmark_common_functions.clustering_analysis(
            adata,
            true_labels = adata.obs['Group'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=n_neighbors,
            n_clusters= adata.obs['Group'].nunique(),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path(fname.split('/')[-1]),'wb') as f:
            pickle.dump(results_dict,f)
        %xdel adata
        gc.collect()

cosine 25 True None None
cosine 50 True None None
cosine 100 True None None
cosine 500 True None None
euclidean 25 True None None
euclidean 50 True None None
euclidean 100 True None None
euclidean 500 True None None


# Run raw datasets

In [5]:
base_folder1 = '../data/benchmark_sensitivity_frontiers/Benchmarking_Clustering_Methods_scRNAseq/raw_data/Rdata_qc/'
base_folder2 = '../data/benchmark_sensitivity_frontiers/Benchmarking_Clustering_Methods_scRNAseq/norm_data/Rdata_qc/'
get_fnames = lambda base_folder: [os.path.join(base_folder, fname) for fname in os.listdir(base_folder)]
fnames = get_fnames(base_folder1)+get_fnames(base_folder2)

resdir = '../results/benchmark_sensitivity_frontiers/real_data/'

#get_res_path_raw = lambda fname: f'{resdir}{fname}_RAW_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn{n_neighbors}_seed{seed}.pkl'
get_res_path_raw = lambda fname: f'{resdir}{fname}_RAW_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}.pkl'
get_res_path_raw_sc = lambda fname: f'{resdir}{fname}_RAW_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}_scanpyhubness.pkl'

In [6]:
#fixed params
do_log = True
do_pca = True
n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = ('duo','seurat')#,'zheng17')
norm_scale = (True,False)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))

In [4]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for fname in fnames:
        if get_res_path_raw(fname).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname.split('/')[-1])
        x = ro.r.load(fname)
        adata = ro.r['sce']    
        if 'kolodziejczyk'in fname or 'Treutlein' in fname:
            adata.obs['Group']=adata.obs['cell_type1']
        results_dict = benchmark_common_functions.clustering_analysis(
            adata,
            true_labels = adata.obs['Group'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            n_clusters= adata.obs['Group'].nunique(),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path_raw(fname.split('/')[-1]),'wb') as f:
            pickle.dump(results_dict,f)
        %xdel adata
        gc.collect()

iteration is used:16 at resolution 2.999997138977051
Cannot find the number of clusters
Clustering solution from last iteration is used:16 at resolution 2.999997138977051
Cannot find the number of clusters
Clustering solution from last iteration is used:15 at resolution 2.999997138977051
Cannot find the number of clusters
Clustering solution from last iteration is used:16 at resolution 2.999997138977051
Cannot find the number of clusters
Clustering solution from last iteration is used:15 at resolution 2.999997138977051
			Hubness methods full pipeline: 0.65 mn
		Zeisel2015.Rdata
		duo norm retained 5000 genes
			Preprocessing done: 0.25 mn
			Inputs generated: 0.21 mn
			Scanpy louvain/leiden clus: 0.16 mn
			Louvain/leiden clus: 0.01 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.34 mn
			Base louvain/leiden clus, searching ground truth: 0.01 mn
			Scoring: 0.0 mn
			Hubness methods full pipeline: 1.55 mn
		Biase2014.Rdata
		duo norm retained 5000 genes
			Preprocessing d

In [7]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for fname in fnames:
        if get_res_path_raw_sc(fname).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname.split('/')[-1])
        x = ro.r.load(fname)
        adata = ro.r['sce']    
        if 'kolodziejczyk'in fname or 'Treutlein' in fname:
            adata.obs['Group']=adata.obs['cell_type1']
        results_dict = benchmark_common_functions.scanpy_hubness_analysis(
            adata,
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path_raw_sc(fname.split('/')[-1]),'wb') as f:
            pickle.dump(results_dict,f)
        %xdel adata
        gc.collect()

cosine 25 True duo True
		Baron2016_m.Rdata


/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


		duo norm retained 5000 genes
			Scoring: 0.34 mn
		Darmanis2015.Rdata
		duo norm retained 5000 genes
			Scoring: 0.19 mn
		Deng2014_raw.Rdata


/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


		duo norm retained 5000 genes
			Scoring: 0.06 mn
		Goolam2016.Rdata


...

		seurat norm retained 4864 genes
			Scoring: 1.63 mn
		Treutlein.Rdata


/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


		seurat norm retained 3854 genes
			Scoring: 0.01 mn
		Yan2013.Rdata


/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


		seurat norm retained 5804 genes
			Scoring: 0.01 mn
